### Lashi el Popular
Lashi ya casi se gradúa y quiere ser popular, para eso es que escogió esta carrera. Planea ser popular creando un grupo solo de gente popular, así el también será popular. En el aula hay varias opiniones de quien no es **"cool"**, por ejemplo: Karel puede pensar que Lashi no es **"cool"** o simplemente no pensar nada, a su vez Lashi puede tener sus propias opiniones.

Como los populares no pueden ser muchos, Lashi tiene que darse a la tarea de crear **el mayor grupo posible de estudiantes populares**, no mayores que un entero positivo $k$, que cumpla que: nadie piensa que alguien del grupo de los populares no es **"cool"**, a no ser que sea otro popular.

### Análisis
En otras palabras, lo que se pide en el ejercicio es encontrar un **grupo maximal de tamaño menor o igual que $k$** en el que **todos** los estudiantes fuera del grupo tengan opinión **neutral** hacia **todos** los que están dentro.

### Modelación

El problema se puede modelar como un grafo $G = (V, E)$ en el que cada nodo representa un estudiante y se cumple que:

 $(u,v) \in E \Longleftrightarrow $ el estudiante que representa el nodo $u$  tiene opinión neutra sobre el estudiante que representa el nodo $v$.

### Conjunto k-dominante
Sea k un entero positivo. Un subconjunto de vértices $D$ de un grafo $G = (V, E)$ es un conjunto k-dominante perfecto de $G$ si cada vértice $v$ de $G$, que no está en $D$, es adyacente a exactamente k vértices de $D$. La cardinalidad mínima de un conjunto k-dominante perfecto de $G$ es el número de k-dominación perfecta γkp(G). En el artículo 
["Perfect k-domination in graphs"](https://ajc.maths.uq.edu.au/pdf/48/ajc_v48_p175.pdf) se demuestra que el problema de encontrar el menor conjunto k-dominante perfecto es NP-completo.

### Relación entre el problema Lashi el Popular y conjunto k-dominante 

Encontrar un conjunto p-dominante perfecto de tamaño p en el problema inicial representa precisamente encontrar un grupo de tamaño p en el que todos los estudiantes fuera del grupo tienen opinión neutra sobre los p que están dentro.

Luego, este problema planteado en términos de conjunto dominante se puede interpretar como obtener el conjunto p-dominate tal que:

$\forall q ~,~ p < q \leq k$ no existe conjunto p-dominante en el grafo.

Por tanto, el problema de decisión asociado consiste en determinar dado un valor $p$ si existe un conjunto p-dominante de tamaño p.
Dado que este también consituye el problema de dicisión asociado a encontrar el menor conjunto k-dominante perfecto, entonces el problema Lashi el popular también es NP-completo.

### Primera Solución (Fuerza Bruta)
La primera propuesta consiste en encontrar el espacio de soluciones válidas para el problema:
* Sea $u,v \in V(G)$ tal que $u \rightarrow v \in E(G)$ que representa **$u$ piensa que $v$ no es cool**, si $v \in P$ donde $P$ es el conjunto popular entonces $u \in P$ también, ya que tiene una mala opinión de $v$ y por ende no puede quedar externo al grupo.  

Realizando este proceso para todos los vértices del grafo, se obtienen todos los conjuntos maximales más pequeños posibles en el que cada uno de los vértices del grafo es partícipe. Para ello se realiza el reverso del grafo tal que por cada vértice se obtenga quién tiene mala opinión suya.

In [1]:
def create_reversed(bad_opinions:list[list[int]]):
    rvsd = [[] for i in bad_opinions]
    for i,bad_opinion in enumerate(bad_opinions):
        for value in bad_opinion:
            rvsd[value].append(i)
    return rvsd

In [2]:
def print_graph(graph:list[list[int]]):
    for i,value in enumerate(graph):
        print(f'{i} --> {value}')

In [3]:
bad_opinions = [[3],[3,5],[4],[0],[0,3],[6],[5]]
print('Bad Opinion Graph')
print_graph(bad_opinions)
print()
reversed = create_reversed(bad_opinions)
print('Bad Opinion Reversed Graph')
print_graph(reversed)

Bad Opinion Graph
0 --> [3]
1 --> [3, 5]
2 --> [4]
3 --> [0]
4 --> [0, 3]
5 --> [6]
6 --> [5]

Bad Opinion Reversed Graph
0 --> [3, 4]
1 --> []
2 --> []
3 --> [0, 1, 4]
4 --> [2]
5 --> [1, 6]
6 --> [5]


El código anterior presenta complejidad temporal $O(V * E)$ ya que por cada vértice se analiza su lista de adyacencia. Posteriormente, se realiza un DFS por cada vértice del grafo para generar grupos populares maximales más pequeños, siendo su complejidad temporal $O(V^2 * E)$.

In [4]:
def solutions_search(reversed:list[list[int]]):
    solutions_set:list[set] = []
    for i in range(len(reversed)):
        visited = [False for i in reversed]
        solution = []
        dfs_visited(i, reversed, visited, solution)
        if set(solution) not in solutions_set: solutions_set.append(set(solution))
    return solutions_set

def dfs_visited(i:int, reversed:list[list[int]], visited:list[bool], solution:list[int]):
    solution.append(i)
    visited[i] = True
    for adj in reversed[i]:
        if not visited[adj]:
            dfs_visited(adj, reversed, visited, solution)

In [5]:
print(solutions_search(create_reversed(bad_opinions)))

[{0, 1, 2, 3, 4}, {1}, {2}, {2, 4}, {1, 5, 6}]


Para la solución se parte, se que toda solución posible en este problema es un conjunto maximal en cuanto a mala opinión (ya que nadie de afuera tiene mala opinión de los de adentro, si hubiese uno entonces el DFS lo hubiese agregado al grupo y caería en contradicción de que se tenía un grupo maximal) o es una mezcla entre grupos maximales. Notar que si dos grupos maximales se mezclan, se mantiene la invariante, es decir el grupo resultante sigue siendo maximal.

![Ejemplo de Grafo](graph.png)

En la imagen anterior los grupos serían: $P = [\{1\}, \{2,6,3,1\}, \{3,1\}, \{4,5,6,3,1\},\{5,6,3,1\}, \{6\}]$  
Una mezcla entre dos grupos cualquiera seguirá siendo maximal en cuanto a opiniones:
* uno es subconjunto del otro, en cuyo caso el mayor abarca las opiniones que pudieron surgir hacia un miembro nuevo;
* son completamente disjuntos, en cuyo caso la unión de los nuevos miembros no afectará las opiniones ya que ambos recogían la mayor opinión negativa de sus respectivos miembros;
* intersección no es vacía, es decir, tienen miembros en común, y la unión de nuevos miembros también recoge la mayor opinión negativa de los miembros que no están en la intersección, ya que los de la intersección son maximales entre sí.  
Ejemplo: $\{1\} + \{6\} = \{1,6\}$ formada por la mezcla de grupos maximales más pequeños.  

Finalmente, se realiza un solución **backtrack** en la que se mezclan grupos y se guarda el mayor o uno de los mayores siempre y cuando sea menor o igual que $k$ de modo que se maximiza el resultado hasta llegar a $k$ de ser posible.

In [6]:
def solve(solutions:list[set[int]], k:int):
    return solve_backtrack(solutions, k, 0, set(), set())

def solve_backtrack(solutions:list[set[int]], k:int, index:int, solution:set, max_temp:set):
    if len(solution) == k: return solution
    if index >= len(solutions): return solution
    
    inter = solution.intersection(solutions[index])
    curr = solution.union(solutions[index])
    if len(curr) <= k:
        solution = solve_backtrack(solutions, k, index+1, curr, max_temp) 
        if len(solution) > len(max_temp): max_temp = solution.copy()
        if len(solution) == k: return solution
        
        curr = curr.difference(solutions[index]).union(inter)
        solution = solve_backtrack(solutions, k, index+1, curr, max_temp)
        if len(solution) > len(max_temp): max_temp = solution.copy()
        if len(solution) == k: return solution 
    else:
        solution = solve_backtrack(solutions, k, index+1, solution, max_temp)
        if len(solution) > len(max_temp): max_temp = solution.copy() 
        if len(solution) == k: return solution
    
    return max_temp      

Al ser un backtrack por fuerza bruta, su complejidad temporal es $O(2^n)$, lo cual es extremadamente costoso para grafos con numerosas cantidades de vértices.

In [7]:
def solution(bad_opinions:list[list[int]], k:int):
    rvsd = create_reversed(bad_opinions)
    solutions_set = solutions_search(rvsd)
    valid_solutions = []
    for sols in solutions_set:
        if len(sols) == k: return sols
        elif len(sols) < k: valid_solutions.append(sols)
    if len(valid_solutions) > 0: return solve(valid_solutions, k)
    return "Not Valid Solution"

In [9]:
bad_opinions = [[3],[3,5],[4],[0],[0,3],[6],[5]]
print('k=1', solution(bad_opinions,1))
print('k=2', solution(bad_opinions,2))
print('k=3', solution(bad_opinions,3))
print('k=4', solution(bad_opinions,4))
print('k=5', solution(bad_opinions,5))
print('k=6', solution(bad_opinions,6))
print('k=7', solution(bad_opinions,7))

k=1 {1}
k=2 {2, 4}
k=3 {1, 5, 6}
k=4 {1, 2, 5, 6}
k=5 {0, 1, 2, 3, 4}
k=6 {0, 1, 2, 3, 4}
k=7 {0, 1, 2, 3, 4, 5, 6}


Notar que para el caso de $k = 6$ no es posile encontrar un grupo tal que el que quede afuera no tenga opinión negativa del resto, por lo que se encontró uno de los grupos (lo mayor posible) con tamaño cercano a $k$.